<a href="https://colab.research.google.com/github/jarimso/Lenguaje-Natural/blob/main/Taller_1_LPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Taller 1.**
---

**Integrantes**: Yasmin Johana Garcia - Javier Ricardo Muñoz Castillo


***Análisis de Sentimientos con Reseñas de productos de Amazon.***

Práctica Integral de NLP con el Dataset "Amazon Reviews"

Este notebook aplica las técnicas de la Sesión 1 (Spacy, NLTK,
análisis de sentimientos) sobre el archivo 'amazonreviews.tsv'.


Descargamos los datos, para esta practica usaremos desde kaggle el dataset **"Amazon Reviews"**

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("yasash/nlp-udemy-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/nlp-udemy-dataset


**Sección 1:** Analisis Exploratorio del Corpus de reseñas


---



1.1: ***Preparación e Importaciones***

In [ ]:
import spacy
from spacy.matcher import Matcher
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Cargar el modelo de Spacy en inglés
nlp = spacy.load('en_core_web_sm')

1.2: ***Carga y Limpieza de los Datos***

In [ ]:
# Cargamos los datos desde el archivo TSV (Tab-Separated Values)
df = pd.read_csv('/root/.cache/kagglehub/datasets/yasash/nlp-udemy-dataset/versions/1/amazonreviews.tsv', sep='\t')
df.columns = ['label', 'review'] # Renombramos las columnas para mayor claridad

# Eliminamos filas con reseñas nulas o vacías
df.dropna(inplace=True)
df = df[df['review'].str.strip() != '']

print(f"\nSe han cargado {len(df)} reseñas del archivo 'amazonreviews.tsv'.")
print("\nEjemplo de los datos:")
print(df.head())


Se han cargado 10000 reseñas del archivo 'amazonreviews.tsv'.

Ejemplo de los datos:
  label                                             review
0   pos  Stuning even for the non-gamer: This sound tra...
1   pos  The best soundtrack ever to anything.: I'm rea...
2   pos  Amazing!: This soundtrack is my favorite music...
3   pos  Excellent Soundtrack: I truly like this soundt...
4   pos  Remember, Pull Your Jaw Off The Floor After He...


In [ ]:
#Revisión de Valanceo
df.label.value_counts()

,count
label,
neg,5097
pos,4903


1.3: ***Procesamiento y Análisis Básico con Spacy***

In [ ]:
# Tomaremos la primera reseña para un análisis detallado.
# Usamos nlp.pipe para procesar textos de forma eficiente, aunque aquí solo es uno.
primera_resena_texto = df['review'].iloc[0]
doc = nlp(primera_resena_texto)

# Conteo básico de tokens y oraciones
tokens_totales = len(doc)
oraciones = list(doc.sents)
oraciones_totales = len(oraciones)

print(f"\n--- Análisis de la primera reseña ---")
print(f"Texto: '{primera_resena_texto}'")
print(f"Número de tokens: {tokens_totales}")
print(f"Número de oraciones: {oraciones_totales}")


--- Análisis de la primera reseña ---
Texto: 'Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^'
Número de tokens: 89
Número de oraciones: 7


In [ ]:
# Análisis detallado de la primera oración de la reseña
primera_oracion = oraciones[0]
print("\nAnálisis detallado de la primera oración:")
print(f"{'Texto':<15} {'POS':<10} {'Dependencia':<15} {'Lema'}")
print("-" * 55)
for token in primera_oracion:
    if not token.is_space:
        print(f"{token.text:<15} {token.pos_:<10} {token.dep_:<15} {token.lemma_}")


Análisis detallado de la primera oración:
Texto           POS        Dependencia     Lema
-------------------------------------------------------
Stuning         VERB       advcl           stun
even            ADV        advmod          even
for             ADP        prep            for
the             DET        det             the
non             NOUN       pobj            non
-               NOUN       pobj            -
gamer           NOUN       pobj            gamer
:               PUNCT      punct           :
This            DET        det             this
sound           NOUN       amod            sound
track           NOUN       nsubj           track
was             AUX        ROOT            be
beautiful       ADJ        acomp           beautiful
!               PUNCT      punct           !


1.4: ***Busqueda de Patrones con Matcher***

In [ ]:
# Buscaremos patrones comunes en reseñas, como "sound track" o "video game".
print("\n--- Búsqueda de patrones con Matcher en la primera reseña ---")
matcher = Matcher(nlp.vocab)

pattern1 = [{'LOWER': 'sound'}, {'LOWER': 'track'}]
pattern2 = [{'LOWER': 'video'}, {'IS_PUNCT': True, 'OP': '?'}, {'LOWER': 'game'}] # El ? hace opcional la puntuación

matcher.add('GameSoundtrackTerms', [pattern1, pattern2])
found_matches = matcher(doc)

if found_matches:
    print(f"Se encontraron {len(found_matches)} patrones de interés:")
    for match_id, start, end in found_matches:
        span = doc[start:end]
        print(f"- Término encontrado: '{span.text}'")
else:
    print("No se encontraron los patrones 'sound track' o 'video game' en la primera reseña.")




--- Búsqueda de patrones con Matcher en la primera reseña ---
Se encontraron 1 patrones de interés:
- Término encontrado: 'sound track'


**Sección 2**: Construcción de un clasificador de sentimientos


---



2.1: Importaciones y División de Datos

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics

# Dividimos el DataFrame completo en conjuntos de entrenamiento y prueba
X = df['review']  # Los textos de las reseñas
y = df['label']   # Las etiquetas 'pos' o 'neg'

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
print(f"\nDatos divididos: {len(X_train)} para entrenamiento, {len(X_test)} para prueba.")


Datos divididos: 7000 para entrenamiento, 3000 para prueba.


2.2: Cración y entrenamiento del Pipeline de Machine Learning

In [ ]:
# Usaremos el mismo pipeline efectivo: TfidfVectorizer + LinearSVC
sentiment_classifier_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

# Entrenamos el modelo con los datos de entrenamiento
print("\nEntrenando el modelo de clasificación de sentimientos...")
sentiment_classifier_pipeline.fit(X_train, y_train)
print("¡Modelo entrenado exitosamente!")


Entrenando el modelo de clasificación de sentimientos...
¡Modelo entrenado exitosamente!


2.3: Evaluación del Desempeño del Modelo con sklearn

In [ ]:
print("\n--- Evaluación del modelo en el conjunto de prueba ---")
predictions = sentiment_classifier_pipeline.predict(X_test)

# Reporte de clasificación con métricas clave
print("Reporte de Clasificación:")
print(metrics.classification_report(y_test, predictions))

# Matriz de confusión para visualizar errores
print("Matriz de Confusión:")
print(metrics.confusion_matrix(y_test, predictions))

# Precisión general del modelo
print(f"Precisión General (Accuracy): {metrics.accuracy_score(y_test, predictions):.4f}")


--- Evaluación del modelo en el conjunto de prueba ---
Reporte de Clasificación:
              precision    recall  f1-score   support

         neg       0.87      0.89      0.88      1518
         pos       0.89      0.86      0.87      1482

    accuracy                           0.88      3000
   macro avg       0.88      0.87      0.87      3000
weighted avg       0.88      0.88      0.87      3000

Matriz de Confusión:
[[1353  165]
 [ 210 1272]]
Precisión General (Accuracy): 0.8750


**Conclusión**: El modelo de clasificación de sentimientos es muy bueno y bastante confiable. Con una Precisión General (Accuracy) del 87.5%, el modelo es capaz de predecir correctamente si una reseña es positiva or negativa en aproximadamente 88 de cada 100 casos que no ha visto antes. Este es un rendimiento sólido para una tarea de clasificación de texto.

***Matriz de Confusión***
Esta matriz nos muestra exactamente dónde se equivoca el modelo:

1353 (Verdaderos Negativos): Acertó al clasificar 1353 reseñas negativas.

1272 (Verdaderos Positivos): Acertó al clasificar 1272 reseñas positivas.

165 (Falsos Positivos): Se equivocó al clasificar 165 reseñas (que eran negativas) como positivas. Este es el "error tipo I".

210 (Falsos Negativos): Se equivocó al clasificar 210 reseñas (que eran positivas) como negativas. Este es el "error tipo II".

2.4: Pruebas con Nuevas Reseñas

In [ ]:
print("\n--- Probando el modelo con ejemplos nuevos ---")
review_positiva = "This is the best product I have ever bought, amazing quality and fast shipping!"
review_negativa = "A complete disappointment. It broke after just one day and the customer service was terrible."

prediccion_pos = sentiment_classifier_pipeline.predict([review_positiva])[0]
prediccion_neg = sentiment_classifier_pipeline.predict([review_negativa])[0]

print(f"Reseña: '{review_positiva}' -> Predicción: {prediccion_pos}")
print(f"Reseña: '{review_negativa}' -> Predicción: {prediccion_neg}")


--- Probando el modelo con ejemplos nuevos ---
Reseña: 'This is the best product I have ever bought, amazing quality and fast shipping!' -> Predicción: pos
Reseña: 'A complete disappointment. It broke after just one day and the customer service was terrible.' -> Predicción: neg


2.5: Evaluación del Desempeño del Modelo con NLTK

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()
df['scores'] = df.review.apply(lambda r: sid.polarity_scores(r))
df.head()

,label,review,scores
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co..."
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co..."
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com..."
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com..."
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp..."


2.4.1: Conversión en etiquetas de predicción

In [ ]:
df['compound'] = df.scores.apply(lambda s: s['compound'])
df['prediction'] = df['compound'].apply(lambda c: 'pos' if c > 0 else 'neg')
df.head()

,label,review,scores,compound,prediction
0,pos,Stuning even for the non-gamer: This sound tra...,"{'neg': 0.088, 'neu': 0.669, 'pos': 0.243, 'co...",0.9454,pos
1,pos,The best soundtrack ever to anything.: I'm rea...,"{'neg': 0.018, 'neu': 0.837, 'pos': 0.145, 'co...",0.8957,pos
2,pos,Amazing!: This soundtrack is my favorite music...,"{'neg': 0.04, 'neu': 0.692, 'pos': 0.268, 'com...",0.9858,pos
3,pos,Excellent Soundtrack: I truly like this soundt...,"{'neg': 0.09, 'neu': 0.615, 'pos': 0.295, 'com...",0.9814,pos
4,pos,"Remember, Pull Your Jaw Off The Floor After He...","{'neg': 0.0, 'neu': 0.746, 'pos': 0.254, 'comp...",0.9781,pos


In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

y_true = df.label.values
y_pred = df.prediction.values

acc = accuracy_score(y_true, y_pred)
cm = confusion_matrix(y_true, y_pred)
cr = classification_report(y_true, y_pred)


print(f"Accuracy:\n{acc}\n")
print(f"Classification Report:\n{cr}")
print(f"Confusion Matrix:\n{cm}")

Accuracy:
0.713

Classification Report:
              precision    recall  f1-score   support

         neg       0.85      0.53      0.65      5097
         pos       0.65      0.90      0.75      4903

    accuracy                           0.71     10000
   macro avg       0.75      0.72      0.70     10000
weighted avg       0.75      0.71      0.70     10000

Confusion Matrix:
[[2716 2381]
 [ 489 4414]]


El modelo NLTK tiene un rendimiento moderado, con una Precisión General (Accuracy) del 71.3%. Si bien es capaz de identificar correctamente el sentimiento en 7 de cada 10 casos, su desempeño es muy desequilibrado y presenta un sesgo significativo. Muestra una clara fortaleza para identificar reseñas positivas, pero a costa de ser muy deficiente para clasificar correctamente las reseñas negativas.

